# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [135]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pylab as pl
import numpy as np


In [119]:
df_raw = pd.read_csv("admissions.csv")
df = df_raw.dropna() 
print(df.head())


   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [120]:
# frequency table for prestige and whether or not someone was admitted
df.groupby(['prestige'])['admit'].value_counts()

prestige  admit
1.0       1        33
          0        28
2.0       0        95
          1        53
3.0       0        93
          1        28
4.0       0        55
          1        12
Name: admit, dtype: int64

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [122]:
dummy_ranks = pd.get_dummies(df.prestige, prefix='prestige')

dummy_ranks.head()

,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,0,1,0
1,0,0,1,0
2,1,0,0,0
3,0,0,0,1
4,0,0,0,1


#### 2.2 When modeling our class variables, how many do we need? 



Answer: n-1, in this case 3

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [123]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
handCalc.head()

,admit,gre,gpa,prestige_1.0,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,0,1,0
1,1,660.0,3.67,0,0,1,0
2,1,800.0,4.00,1,0,0,0
3,1,640.0,3.19,0,0,0,1
4,0,520.0,2.93,0,0,0,1


In [124]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
crosstab_freq = pd.crosstab(handCalc['prestige_1.0'], handCalc.admit,margins=True)
crosstab_freq

admit,0,1,All
prestige_1.0,,,
0,243,93,336
1,28,33,61
All,271,126,397


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [125]:
crosstab_freq.iloc[1,2]

61

In [126]:
format(crosstab_freq.iloc[1,1] / crosstab_freq.iloc[1,2],'0.2%')

'54.10%'

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [127]:
format((crosstab_freq.iloc[0,1]) / crosstab_freq.iloc[0,2],'0.2%')

'27.68%'

#### 3.3 Calculate the odds ratio

In [128]:
(33/28)/(93/243)

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: Applicants who came from a #1 ranked college were 3 times as likely to be admited than those who did not.

#### 3.5 Print the cross tab for prestige_4

In [129]:
crosstab_freq = pd.crosstab(handCalc['prestige_4.0'], handCalc.admit,margins=True)
crosstab_freq

admit,0,1,All
prestige_4.0,,,
0,216,114,330
1,55,12,67
All,271,126,397


#### 3.6 Calculate the OR 

In [130]:
((12/55)/(114/216))

0.4133971291866028

#### 3.7 Write this finding in a sentence

Answer: Students who came from the lowest ranked school were less than half as likely to get admitted.

## Part 4. Analysis

In [131]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
data.head()

,admit,gre,gpa,prestige_2.0,prestige_3.0,prestige_4.0
0,0,380.0,3.61,0,1,0
1,1,660.0,3.67,0,1,0
2,1,800.0,4.00,0,0,0
3,1,640.0,3.19,0,0,1
4,0,520.0,2.93,0,0,1


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [132]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [167]:
train_cols = ['gre', 'gpa', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0' ]

#### 4.2 Fit the model

In [172]:


results = sm.OLS(data['admit'], data[train_cols]).fit()

#### 4.3 Print the summary results

In [173]:
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  admit   R-squared:                       0.383
Model:                            OLS   Adj. R-squared:                  0.375
Method:                 Least Squares   F-statistic:                     48.68
Date:                Mon, 14 Aug 2017   Prob (F-statistic):           3.83e-39
Time:                        12:54:46   Log-Likelihood:                -239.63
No. Observations:                 397   AIC:                             489.3
Df Residuals:                     392   BIC:                             509.2
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
gre              0.0004      0.000      1.812      0.071   -3.19e-05       0.001
gpa              0.0950      0.039      2.429      0.016       0.018       0.172
prestige_2.0    -0.1854      0.065     -2.855      0.005      -0.313      -0.058
prestige_3.0    -0.3102      0.068     -4.557      0.000      -0.444      -0.176
prestige_4.0    -0.3499      0.076     -4.615      0.000      -0.499      -0.201
==============================================================================
Omnibus:                      161.246   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               52.229
Skew:                           0.695   Prob(JB):                     4.56e-12
Kurtosis:                       1.893   Cond. No.                     2.80e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.8e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [177]:
np.exp(results.params)

gre             1.000375
gpa             1.099693
prestige_2.0    0.830745
prestige_3.0    0.733289
prestige_4.0    0.704786
dtype: float64

In [181]:
conf = results.conf_int()
conf['OR']= results.params
conf.columns=['2.5%', '97.5%', 'OR']
np.exp(conf)

,2.5%,97.5%,OR
gre,0.999968,1.000782,1.000375
gpa,1.018285,1.187610,1.099693
prestige_2.0,0.731163,0.943888,0.830745
prestige_3.0,0.641423,0.838311,0.733289
prestige_4.0,0.607202,0.818052,0.704786


#### 4.5 Interpret the OR of Prestige_2

Answer: An applicant has has lower odds of being admitted because the odds ratio is less than 1

#### 4.6 Interpret the OR of GPA

Answer: An applicant has higher odds of being admitted because the odds ratio is close to/greater than 1

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [234]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    #m = n / arrays[0].size
    m = n // arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in range(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [235]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print(gres)
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print(gpas)
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [236]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'admit']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1.0', 'prestige_2.0', 'prestige_3.0', 'prestige_4.0']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'admit']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])


#### 5.2 Make predictions on the enumerated dataset

In [238]:
combos['admission']=results.predict(combos[train_cols])
combos.tail(4)

,gre,gpa,prestige,admit,prestige_2.0,prestige_3.0,prestige_4.0,pred,admission
396,800.0,4.0,1.0,1.0,0,0,0,0.679861,0.679861
397,800.0,4.0,2.0,1.0,1,0,0,0.494428,0.494428
398,800.0,4.0,3.0,1.0,0,1,0,0.369645,0.369645
399,800.0,4.0,4.0,1.0,0,0,1,0.330000,0.330000


#### 5.3 Interpret findings for the last 4 observations

Answer: the probability of admission is higher for students with higher ranked undergraduate schools.

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.

In [292]:
group_data=pd.pivot_table(combos, 'admission', ['gre','gpa'], 'prestige')   ### work in Progress
group_data

prestige                  1.0       2.0       3.0       4.0
gre        gpa                                             
220.000000 2.260000  0.297198  0.111765 -0.013018 -0.052663
           2.453333  0.315571  0.130138  0.005355 -0.034290
           2.646667  0.333944  0.148511  0.023728 -0.015917
           2.840000  0.352316  0.166883  0.042100  0.002455
           3.033333  0.370689  0.185256  0.060473  0.020828
           3.226667  0.389062  0.203629  0.078846  0.039201
           3.420000  0.407434  0.222002  0.097219  0.057573
           3.613333  0.425807  0.240374  0.115591  0.075946
           3.806667  0.444180  0.258747  0.133964  0.094319
           4.000000  0.462553  0.277120  0.152337  0.112692
284.444444 2.260000  0.321343  0.135911  0.011128 -0.028518
           2.453333  0.339716  0.154283  0.029500 -0.010145
           2.646667  0.358089  0.172656  0.047873  0.008228
           2.840000  0.376462  0.191029  0.066246  0.026601
           3.033333  0.394834  0.209401  0.084619  0.044973
           3.226667  0.413207  0.227774  0.102991  0.063346
           3.420000  0.431580  0.246147  0.121364  0.081719
           3.613333  0.449952  0.264520  0.139737  0.100092
           3.806667  0.468325  0.282892  0.158109  0.118464
           4.000000  0.486698  0.301265  0.176482  0.136837
348.888889 2.260000  0.345489  0.160056  0.035273 -0.004372
           2.453333  0.363862  0.178429  0.053646  0.014001
           2.646667  0.382234  0.196801  0.072018  0.032373
           2.840000  0.400607  0.215174  0.090391  0.050746
           3.033333  0.418980  0.233547  0.108764  0.069119
           3.226667  0.437352  0.251920  0.127137  0.087491
           3.420000  0.455725  0.270292  0.145509  0.105864
           3.613333  0.474098  0.288665  0.163882  0.124237
           3.806667  0.492471  0.307038  0.182255  0.142610
           4.000000  0.510843  0.325410  0.200627  0.160982
...                       ...       ...       ...       ...
671.111111 2.260000  0.466216  0.280783  0.156000  0.116355
           2.453333  0.484588  0.299155  0.174372  0.134727
           2.646667  0.502961  0.317528  0.192745  0.153100
           2.840000  0.521334  0.335901  0.211118  0.171473
           3.033333  0.539706  0.354274  0.229491  0.189845
           3.226667  0.558079  0.372646  0.247863  0.208218
           3.420000  0.576452  0.391019  0.266236  0.226591
           3.613333  0.594825  0.409392  0.284609  0.244964
           3.806667  0.613197  0.427764  0.302982  0.263336
           4.000000  0.631570  0.446137  0.321354  0.281709
735.555556 2.260000  0.490361  0.304928  0.180145  0.140500
           2.453333  0.508734  0.323301  0.198518  0.158873
           2.646667  0.527106  0.341673  0.216891  0.177245
           2.840000  0.545479  0.360046  0.235263  0.195618
           3.033333  0.563852  0.378419  0.253636  0.213991
           3.226667  0.582225  0.396792  0.272009  0.232364
           3.420000  0.600597  0.415164  0.290381  0.250736
           3.613333  0.618970  0.433537  0.308754  0.269109
           3.806667  0.637343  0.451910  0.327127  0.287482
           4.000000  0.655715  0.470283  0.345500  0.305854
800.000000 2.260000  0.514506  0.329073  0.204290  0.164645
           2.453333  0.532879  0.347446  0.222663  0.183018
           2.646667  0.551252  0.365819  0.241036  0.201391
           2.840000  0.569624  0.384192  0.259409  0.219763
           3.033333  0.587997  0.402564  0.277781  0.238136
           3.226667  0.606370  0.420937  0.296154  0.256509
           3.420000  0.624743  0.439310  0.314527  0.274882
           3.613333  0.643115  0.457682  0.332900  0.293254
           3.806667  0.661488  0.476055  0.351272  0.311627
           4.000000  0.679861  0.494428  0.369645  0.330000

[100 rows x 4 columns]

In [293]:
plot_data_index = group_data.set_index('gre', 'gpa').sort_index()

KeyError: 'gre'

In [282]:
def admit_plot(variable):
    data = group_data.loc[variable]
    plt.plot(data.index, data.values)

In [280]:
admit_plot('gre')

KeyError: 'the label [gre] is not in the [index]'